# 并行和分布式机器学习

[Dask-ML](https://dask-ml.readthedocs.io)有并行和分布式机器学习的资源。

## 问题规模类型

你可能会面临几个不同的缩放问题。
缩放策略取决于你面对的是哪个问题。

1. CPU限制。数据适合在RAM中，但训练时间太长。许多超参数组合，许多模型的大集合等。
2. 内存约束：数据大于RAM，不能选择采样。数据比RAM大，抽样不是一个选项。

![](images/ml-dimensions.png)

* 对于内存问题，只需使用scikit-learn（或你喜欢的ML库）。
* 对于大型模型，使用`dask_ml.joblib`和你喜欢的scikit-learn估计器。
* 对于大型数据集，使用`dask_ml`估计器。

## Scikit-5分钟内学会

Scikit-Learn有一个不错的、一致的API。

1. 你实例化一个 "估计器"（如 "线性回归"、"随机森林分类器 "等）。所有的模型*hyperparameters*（用户指定的参数，而不是估计器学习的参数）都会在创建时传递给估计器。
2. 你调用`estimator.fit(X，y)`来训练估计器。
3. 使用`estimator`来检查属性，进行预测等。

让我们产生一些随机数据。

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10000, n_features=4, random_state=0)
X[:8]

In [ ]:
y[:8]

我们将拟合一个支持向量分类器。

In [ ]:
from sklearn.svm import SVC

创建估计器，并对其进行拟合。

In [ ]:
estimator = SVC(random_state=0)
estimator.fit(X, y)

检查所学的属性。

In [ ]:
estimator.support_vectors_[:4]

检查准确度。

In [ ]:
estimator.score(X, y)

## 超参数

大多数模型都有*超参数*。它们会影响拟合度，但会在前面指定，而不是在训练过程中学习。

In [ ]:
estimator = SVC(C=0.00001, shrinking=False, random_state=0)
estimator.fit(X, y)
estimator.support_vectors_[:4]

In [ ]:
estimator.score(X, y)

## 超参数优化

有几种方法可以在训练时学习到最好的*超*参数。一种是`GridSearchCV`。
顾名思义，这是在超参数组合的网格上进行蛮力搜索。

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
estimator = SVC(gamma='auto', random_state=0, probability=True)
param_grid = {
    'C': [0.001, 10.0],
    'kernel': ['rbf', 'poly'],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2)
grid_search.fit(X, y)

## 用scikit-learn实现单机并行化

![](images/unmerged_grid_search_graph.svg)

Scikit-Learn有很好的*单机*并行性，通过Joblib。
任何可以并行操作的scikit-learn估计器都会暴露一个`n_jobs`关键字。
这控制了将被使用的CPU核数。

In [ ]:
%%time
grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=2, n_jobs=-1)
grid_search.fit(X, y)

## 使用Dask进行多机并行

![](images/merged_grid_search_graph.svg)

Dask可以与scikit-learn对话（通过joblib），这样你的*集群*就会被用来训练一个模型。

如果你在笔记本电脑上运行这个，会花费相当多的时间，但在这段时间内，CPU的使用率会令人满意地接近100%。为了运行得更快，你需要一个disrtibuted集群。这意味着在调用 "Client "的时候加入一些内容，例如

```
c = Client('tcp://my.scheduler.address:8786')
```

关于创建集群的多种方法的细节可以在[这里](https://docs.dask.org/en/latest/setup/single-distributed.html)找到。

让我们在更大的问题上试试（更多的超参数）。

In [ ]:
import joblib
import dask.distributed

c = dask.distributed.Client()

In [ ]:
param_grid = {
    'C': [0.001, 0.1, 1.0, 2.5, 5, 10.0],
    # Uncomment this for larger Grid searches on a cluster
    # 'kernel': ['rbf', 'poly', 'linear'],
    # 'shrinking': [True, False],
}

grid_search = GridSearchCV(estimator, param_grid, verbose=2, cv=5, n_jobs=-1)

In [ ]:
%%time
with joblib.parallel_backend("dask", scatter=[X, y]):
    grid_search.fit(X, y)

In [ ]:
grid_search.best_params_, grid_search.best_score_

# 大型数据集的训练

有时你会想在一个大于内存的数据集上进行训练。`dask-ml`已经实现了估计器，它在dask数组和数据框上工作得很好，这些数据框可能比你的机器的RAM大。

In [ ]:
import dask.array as da
import dask.delayed
from sklearn.datasets import make_blobs
import numpy as np

我们将使用scikit-learn在本地制作一个小型（随机）数据集。

In [ ]:
n_centers = 12
n_features = 20

X_small, y_small = make_blobs(n_samples=1000, centers=n_centers, n_features=n_features, random_state=0)

centers = np.zeros((n_centers, n_features))

for i in range(n_centers):
    centers[i] = X_small[y_small == i].mean(0)
    
centers[:4]

小数据集将是我们大型随机数据集的模板。
我们将使用`dask.delayed`来适配`sklearn.datasets.make_blobs`，这样实际的数据集就会在我们的工人身上生成。

In [ ]:
n_samples_per_block = 200000
n_blocks = 500

delayeds = [dask.delayed(make_blobs)(n_samples=n_samples_per_block,
                                     centers=centers,
                                     n_features=n_features,
                                     random_state=i)[0]
            for i in range(n_blocks)]
arrays = [da.from_delayed(obj, shape=(n_samples_per_block, n_features), dtype=X.dtype)
          for obj in delayeds]
X = da.concatenate(arrays)
X

In [ ]:
X = X.persist()  # Only run this on the cluster.

Dask-ML中实现的算法是可扩展的。它们可以很好地处理大于内存的数据集。

它们遵循 scikit-learn API，所以如果您熟悉 scikit-learn，您会对 Dask-ML 感到宾至如归。

In [ ]:
from dask_ml.cluster import KMeans

In [ ]:
clf = KMeans(init_max_iter=3, oversampling_factor=10)

In [ ]:
%time clf.fit(X)

In [ ]:
clf.labels_

In [ ]:
clf.labels_[:10].compute()